# L1: Language modelling

In this lab you will implement and train two neural language models: the fixed-window model and the recurrent neural network model. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [3]:
import torch

For this lab, you should use the GPU if you have one:

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

## Data

The data for this lab is [WikiText](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the “Good” and “Featured” articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the “training” and the “validation” section of the data. We fill this container by reading the corresponding text files. The only processing we do is to whitespace-tokenise, and to enclose each non-empty line within `<bos>` (beginning-of-sentence) and `<eos>` (end-of-sentence) tokens.

In [5]:
class WikiText(object):
    
    def __init__(self):
        self.vocab = {}
        self.train = self.read_data('wiki.train.tokens')
        self.valid = self.read_data('wiki.valid.tokens')
    
    def read_data(self, path):
        ids = []
        with open(path, encoding='utf-8') as source:
            for line in source:
                line = line.rstrip()
                if line:
                    for token in ['<bos>'] + line.split() + ['<eos>']:
                        if token not in self.vocab:
                            self.vocab[token] = len(self.vocab)
                        ids.append(self.vocab[token])
        return ids

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [6]:
wikitext = WikiText()

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.vocab))

# print('First 10 tokens:', wikitext.vocab)

Tokens in train: 2099444
Tokens in valid: 218808
Vocabulary size: 33279


## Problem 1: Fixed-window model

In this section, you will implement and train the fixed-window neural language model proposed by [Bengio et al. (2003)](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) and presented in the lectures. Recall that an input to the network takes the form of a vector of $n-1$ integers representing the preceding words. Each integer is mapped to a vector via an embedding layer. (All positions share the same embedding.) The embedding vectors are then concatenated and sent through a two-layer feed-forward network with a non-linearity in the form of a rectified linear unit (ReLU) and a final softmax layer.

### Problem 1.1: Vectorise the data (1&nbsp;point)

Your first task is to write code for transforming the data in the WikiText container into a vectorised form that can be fed to the fixed-window model. Concretely, you will implement a [collate function](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) in the form of a callable vectoriser object. Complete the skeleton code in the cell below:

In [7]:
class FixedWindowVectorizer(object):
    def __init__(self, n):
        # n-gram order
        self.n = n

    def __call__(self, data):
        N = len(data) - self.n + 1
        X = torch.zeros(N, self.n-1, dtype=torch.long)
        y = torch.zeros(N, dtype=torch.long)
        for i in range(N):
            X[i] = torch.tensor(data[i:i+self.n-1])
            y[i] = torch.tensor(data[i+self.n-1])
        return X, y

Your code should implement the following specification:

**__init__** (*self*, *n*)

> Creates a new vectoriser with n-gram order $n$. Your code should be able to handle arbitrary n-gram orders $n \geq 1$.

**__call__** (*self*, *data*)

> Transforms WikiText *data* (a list of word ids) into a pair of tensors $\mathbf{X}$, $\mathbf{y}$ that can be used to train the fixed-window model. Let $N$ be the total number of $n$-grams from the token list; then $\mathbf{X}$ is a matrix with shape $(N, n-1)$ and $\mathbf{y}$ is a vector with length $N$.

#### 🤞 Test your code

Test your implementation by running the code in the next cell. Does the output match your expectation?

In [8]:
valid_x, valid_y = FixedWindowVectorizer(3)(wikitext.valid)

print(wikitext.valid[-10:])

print(valid_x)
print(valid_y)

print(valid_x.size(), valid_y.size())

[0, 1, 1, 1, 8628, 4250, 1, 1, 1, 5]
tensor([[    0,     1],
        [    1, 32967],
        [32967, 32968],
        ...,
        [ 4250,     1],
        [    1,     1],
        [    1,     1]])
tensor([32967, 32968,     1,  ...,     1,     1,     5])
torch.Size([218806, 2]) torch.Size([218806])


### Problem 1.2: Implement the model (2&nbsp;points)

Your next task is to implement the fixed-window model based on the graphical specification given in the lecture.

In [9]:
import torch.nn as nn

class FixedWindowModel(nn.Module):

    def __init__(self, n, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.n = n
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.linear = nn.Linear((n-1)*embedding_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, n_words)
        

    def forward(self, x) -> torch.Tensor:
        embedding_matrix = self.embedding(x) # size: batch_size x (n-1) x embedding_dim
        embedded_vectors = embedding_matrix.view(embedding_matrix.size(0), -1)
        output1 = self.linear(embedded_vectors)
        output1_relu = torch.relu(output1)
        output2 = self.linear2(output1_relu)
        return output2


Here is the specification of the two methods:

**__init__** (*self*, *n*, *n_words*, *embedding_dim*=50, *hidden_dim*=50)

> Creates a new fixed-window neural language model. The argument *n* specifies the model&rsquo;s $n$-gram order. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the hidden layer of the feedforward network, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, n-1)$, where $B$ is the batch size. The output of the forward pass is a tensor of shape $(B, V)$ where $V$ is the number of words in the vocabulary.

**Hint:** The most efficient way to implement the vector concatenation in this model is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

In [10]:
m = FixedWindowModel(3, len(wikitext.vocab), 5, 5)

# copy valid x  
print(len(valid_x))

m.forward(valid_x[:3200])

218806


tensor([[-0.0193,  0.4792, -0.1013,  ...,  0.0490, -0.1986,  0.0358],
        [ 0.1811,  0.7492, -0.0484,  ..., -0.2014, -0.5691, -0.0172],
        [ 0.0551,  0.5109, -0.0046,  ...,  0.0653, -0.5866,  0.1615],
        ...,
        [-0.1210,  0.4627,  0.1836,  ...,  0.2146, -0.4578,  0.1747],
        [-0.0667,  0.4666,  0.0313,  ...,  0.1162, -0.3100,  0.0820],
        [ 0.1420,  0.6584, -0.0618,  ..., -0.0383, -0.7639,  0.2747]],
       grad_fn=<AddmmBackward0>)

### Problem 1.3: Train the model (3&nbsp;points)

Your final task is to write code to train the fixed-window model using minibatch gradient descent and the cross-entropy loss function. This should be a straightforward generalisation of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [31]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tqdm


def train_fixed_window(n, n_epochs=1, batch_size=3200, lr=1e-2) -> FixedWindowModel:
    # create model
    model = FixedWindowModel(n, len(wikitext.vocab))

    # init optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # init data loader
    data_loader = DataLoader(
        wikitext.train,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=FixedWindowVectorizer(n),
    )
    for _ in range(n_epochs):
        with tqdm.tqdm(data_loader) as pbar:
            for batch_x, batch_y in pbar:
                # zero gradients
                optimizer.zero_grad()

                # forward pass
                output = model.forward(batch_x)
                # print(f"Output size {output.size()}")
                # print(f"Batch y size {batch_y.size()}")

                # compute loss
                loss = F.cross_entropy(output, batch_y)

                # backward pass
                loss.backward()

                # update parameters
                pbar.set_description_str(f"loss: {loss.item():.4f}")

                # update parameters
                optimizer.step()
        # print perplexity
        print(f"Perplexity: {np.exp(loss.item()):.2f}")
    return model


def validate_fixed_window(model: FixedWindowModel, n_epochs=1, batch_size=3200):
    # init data loader
    data_loader = DataLoader(
        wikitext.valid,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=FixedWindowVectorizer(3),
    )
    with tqdm.tqdm(data_loader) as pbar:
        for _ in range(n_epochs):
            total_loss = 0
            for batch_x, batch_y in pbar:
                with torch.no_grad():
                    # forward pass
                    output = model.forward(batch_x)

                    # compute loss
                    loss = F.cross_entropy(output, batch_y)
                    total_loss += loss.item()
                    pbar.set_description_str(f"loss: {loss.item():.4f}")
            # print perplexity
            print(f"Perplexity: {np.exp(loss.item()):.2f}")
            print(f"Average Perplexity: {np.exp(total_loss/len(data_loader)):.2f}")

Here is the specification of the training function:

**train_fixed_window** (*n*, *n_epochs* = 1, *batch_size* = 3200, *lr* = 0.01)

> Trains a fixed-window neural language model of order *n* using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

The code in the cell below trains a bigram model.

In [34]:
model_fixed_window = train_fixed_window(3, 2)

  0%|          | 0/657 [00:00<?, ?it/s]

loss: 6.2864: 100%|██████████| 657/657 [04:41<00:00,  2.34it/s]


Perplexity: 537.20


loss: 5.5681: 100%|██████████| 657/657 [04:50<00:00,  2.26it/s]

Perplexity: 261.94


In [32]:
validate_fixed_window(model_fixed_window, 2)

loss: 5.6170: 100%|██████████| 69/69 [00:13<00:00,  5.07it/s]


Perplexity: 275.06
Average Perplexity: 326.64
Perplexity: 275.06
Average Perplexity: 326.64


#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 350.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

⚠️ Computing the validation perplexity in one go (for the full validation set) will probably exhaust your computer’s memory and/or take a lot of time. If you run into this problem, do the computation at the minibatch level and aggregate the results.

#### 🤞 Test your code

To see whether your network is learning something, print or plot the loss and/or the perplexity on the training data. If the two values do not decrease during training, try to find the problem before wasting time (and electricity) on useless computation.

Training and even evaluation will take some time – on a CPU, you should expect several minutes per epoch, depending on hardware. Our reference implementation uses a GPU and runs in less than 30 seconds per epoch on [Colab](http://colab.research.google.com).

## Problem 2: Recurrent neural network model

In this section, you will implement the recurrent neural network language model. Recall that an input to this model is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer representing the probability distribution over the words at position $i+1$. In theory, the input vector could represent the complete training data; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*. This length is called the **backpropagation-through-time horizon**.

### Problem 2.1: Vectorise the data (1&nbsp;point)

As in the previous problem, your first task is to transform the data in the WikiText container into a vectorised form that can be fed to the model.

In [12]:
class RNNVectorizer(object):
    def __init__(self, bptt_len: int):
        # backpropagation-through-time horizon
        self.bptt_len = bptt_len

    def __call__(self, data):
        # split data into batches
        N = len(data) - self.bptt_len # +1 TODO: ask how to wrap Y
        X = torch.zeros(N, self.bptt_len, dtype=torch.long)
        Y = torch.zeros(N, self.bptt_len,dtype=torch.long)
        for i in range(N):
            X[i] = torch.tensor(data[i:i+self.bptt_len])
            Y[i] = torch.tensor(data[i+1:i+self.bptt_len+1])
        return X, Y

Your vectoriser should meet the following specification:

**__init__** (*self*, *bptt_len*)

> Creates a new vectoriser. The parameter *bptt_len* specifies the backpropagation-through-time horizon.

**__call__** (*self*, *data*)

> Transforms a list of token indexes *data* into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [13]:
valid_x_rnn, valid_y_rnn = RNNVectorizer(32)(wikitext.valid)

print(valid_x_rnn.size(), valid_y_rnn.size())

torch.Size([218776, 32]) torch.Size([218776, 32])


### Problem 2.2: Implement the model (2&nbsp;points)

Your next task is to implement the recurrent neural network model based on the graphical specification.

In [14]:
import torch.nn as nn

class RNNModel(nn.Module):
    
    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.n_words = n_words
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, n_words)


    def forward(self, x) -> torch.Tensor:
        embedded_x: torch.Tensor = self.embedding(x)
        output1, _ = self.lstm(embedded_x)
        output2: torch.Tensor = self.linear(output1)
        return output2

Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 50, *hidden_dim* = 50)

> Creates a new recurrent neural network language model based on an LSTM. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

In [15]:
# Test RNNModel
V = len(wikitext.vocab)
rnn_model = RNNModel(V)

rnn_model.forward(valid_x_rnn[:3200])

tensor([[[-4.6521e-02,  5.1958e-02, -1.3905e-01,  ...,  7.2335e-02,
           1.0395e-01,  2.1993e-01],
         [-1.4109e-01,  1.7189e-02, -1.6950e-01,  ...,  7.0523e-02,
          -9.5192e-02,  6.7265e-02],
         [ 7.8084e-02, -1.9063e-02, -1.4843e-01,  ...,  1.8883e-01,
           1.4447e-01,  1.9114e-01],
         ...,
         [-1.8073e-02,  8.5087e-03, -7.7400e-02,  ...,  1.4571e-01,
           1.6155e-01,  9.6175e-02],
         [ 2.8605e-02, -4.4566e-02, -1.1397e-01,  ...,  1.7556e-02,
           1.4853e-02,  7.2751e-03],
         [ 1.8843e-04,  2.1162e-02,  7.0548e-02,  ...,  1.3855e-01,
           2.6426e-01,  1.8742e-01]],

        [[-1.3358e-01,  5.5397e-02, -1.5265e-01,  ...,  4.2290e-02,
          -9.1591e-02,  1.0849e-01],
         [ 2.7867e-02, -5.1223e-03, -1.5239e-01,  ...,  1.5352e-01,
           5.0856e-02,  1.3893e-01],
         [ 1.4050e-01,  1.9125e-01, -1.4290e-01,  ...,  7.3667e-02,
           1.8727e-01,  2.3714e-01],
         ...,
         [ 1.4725e-02, -1

### Problem 2.3: Train the model (3&nbsp;points)

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [36]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tqdm


def train_rnn(n_epochs=1, batch_size=1000, bptt_len=32, lr=1e-2):
    # create model
    model = RNNModel(len(wikitext.vocab))

    # init optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # init data loader
    data_loader = DataLoader(
        wikitext.train,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=RNNVectorizer(bptt_len),
    )
    for _ in range(n_epochs):
        with tqdm.tqdm(data_loader) as pbar:
            for batch_x, batch_y in pbar:
                # zero gradients
                optimizer.zero_grad()

                # forward pass
                output = model.forward(batch_x)
                #print(f"Output size {output.size()}")
                #print(f"Batch y size {batch_y.size()}")

                formatted_batch_y = batch_y.view(batch_y.size(0) * batch_y.size(1))
                formatted_output = output.view(output.size(0) * output.size(1), output.size(2))
                #print(f"Formatted batch y size {formatted_batch_y.size()}")
                # compute loss
                loss = F.cross_entropy(
                    formatted_output,
                    formatted_batch_y,
                )

                # backward pass
                loss.backward()

                # update parameters
                pbar.set_description_str(f"loss: {loss.item():.4f}")

                # update parameters
                optimizer.step()
        # print perplexity
        print(f"Perplexity: {np.exp(loss.item()):.2f}")
    return model

Here is the specification of the training function:

**train_rnn** (*n_epochs* = 1, *batch_size* = 1000, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [1]:
model_rnn = train_rnn(bptt_len=5)

NameError: name 'train_rnn' is not defined

#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 280.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

## Problem 3: Parameter initialisation (3&nbsp;points)

The error surfaces explored when training neural networks can be very complex. Because of this, it is important to choose “good” initial values for the parameters. In PyTorch, the weights of the embedding layer are initialised by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the initialisation affects the perplexity of your feed-forward language model. Find research articles that propose different initialisation strategies.

Write a short (150&nbsp;words) report about your experiments and literature search. Use the following prompts:

* What different initialisation did you try? What results did you get?
* How do your results compare to what was suggested by the research articles?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

You are allowed to consult sources for this problem if you appropriately cite them. If in doubt, please read the [Academic Integrity Policy](https://www.ida.liu.se/~TDDE09/logistics/policies.html#academic-integrity-policy).

*TODO: Enter your text here*